# Data Preprocessing

# --- Step 1: Data Cleaning (Pre-Pipeline) ---

Cleaning functions before split:
1. Clean data types:
    - Monthly Debt - remove non-numeric characters, then transform strings to floats.
    - Maximum Open Credit - remove non-numeric characters, then transform strings into floats.
2. Correct inconsistent values:
    -  Home Ownership - change "HaveMortgage" to "Home Mortgage"
3. Remove null values
4. Drop bad values
5. Remove Duplicates

In [157]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn import set_config
set_config(transform_output='pandas')

import category_encoders as ce

In [158]:
df = pd.read_csv('LoansDataset.csv')
df.head()

/var/folders/bh/crnltg950_n8l07mfm97qdtc0000gn/T/ipykernel_6978/2955071538.py:1: DtypeWarning: Columns (10,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('LoansDataset.csv')


,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,6cf51492-02a2-423e-b93d-676f05b9ad53,7c202b37-2add-44e8-9aea-d5b119aea935,Charged Off,12232,Short Term,7280.0,< 1 year,Rent,46643.0,Debt Consolidation,777.39,18.0,10.0,12,0,6762,7946,0.0,0.0
1,552e7ade-4292-4354-9ff9-c48031697d72,e7217b0a-07ac-47dd-b379-577b5a35b7c6,Charged Off,25014,Long Term,7330.0,10+ years,Home Mortgage,81099.0,Debt Consolidation,892.09,26.7,NaN,14,0,35706,77961,0.0,0.0
2,9b5e32b3-8d76-4801-afc8-d729d5a2e6b9,0a62fc41-16c8-40b5-92ff-9e4b763ce714,Charged Off,16117,Short Term,7240.0,9 years,Home Mortgage,60438.0,Home Improvements,1244.02,16.7,32.0,11,1,11275,14815,1.0,0.0
3,5419b7c7-ac11-4be2-a8a7-b131fb6d6dbe,30f36c59-5182-4482-8bbb-5b736849ae43,Charged Off,11716,Short Term,7400.0,3 years,Rent,34171.0,Debt Consolidation,990.94,10.0,NaN,21,0,7009,43533,0.0,0.0
4,1450910f-9495-4fc9-afaf-9bdf4b9821df,70c26012-bba5-42c0-8dcb-75295ada31bb,Charged Off,9789,Long Term,6860.0,10+ years,Home Mortgage,47003.0,Home Improvements,503.71,16.7,25.0,13,1,16913,19553,1.0,0.0


### Step 1: Data Cleaning - 1. Clean data types:
    - Monthly Debt - remove non-numeric characters, then transform strings to floats.
    - Maximum Open Credit - remove non-numeric characters, then transform strings into floats.

In [197]:
def string_to_float(data, column_name):
  data = data.copy()
  data[column_name] = data[column_name].astype(str)
  data[column_name] = data[column_name].str.replace(r'[^0-9.]', '', regex=True)
  data[column_name] = data[column_name].replace('', np.nan)
  data[column_name] = pd.to_numeric(data[column_name], errors='coerce')
  return data

In [198]:
monthly_debt_to_float_transformer = FunctionTransformer(lambda X: string_to_float(X, 'Monthly Debt'))
max_credit_to_float_tranformer = FunctionTransformer(lambda X: string_to_float(X, 'Maximum Open Credit'))

In [ ]:
df = string_to_float(df, 'Monthly Debt')
df = string_to_float(df, 'Maximum Open Credit')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111107 entries, 0 to 111106
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Loan ID                       111107 non-null  object 
 1   Customer ID                   111107 non-null  object 
 2   Loan Status                   111107 non-null  object 
 3   Current Loan Amount           111107 non-null  int64  
 4   Term                          111107 non-null  object 
 5   Credit Score                  89769 non-null   float64
 6   Years in current job          106414 non-null  object 
 7   Home Ownership                111107 non-null  object 
 8   Annual Income                 89769 non-null   float64
 9   Purpose                       111107 non-null  object 
 10  Monthly Debt                  111107 non-null  float64
 11  Years of Credit History       111107 non-null  float64
 12  Months since last delinquent  52104 non-null

### Step 1: Data Cleaning - 2. Correct inconsistent values:
    -  Home Ownership - change "HaveMortgage" to "Home Mortgage"

In [161]:
def correct_label(data, column_name, wrong_label, right_label):
  """
  data = your dataframe
  column_name = column name as a string
  the incorrect label that you want to change
  the correct label that you want to change it to
  """
  data[column_name] = data[column_name].replace({wrong_label: right_label})
  return data

In [162]:
correct_label(df, 'Home Ownership', 'HaveMortgage', 'Home Mortgage')
df['Home Ownership'].unique()

array(['Rent', 'Home Mortgage', 'Own Home'], dtype=object)

### Step 1: Data Cleaning - 3. Drop nulls
Drop null values in the following features:
   - Credit Score
   - Annual Income
   - Bankruptcies
   - Tax Liens

In [163]:
def drop_nulls(data, columns_list):
  data = data.copy()
  return data.dropna(subset=columns_list)

In [164]:
columns=['Credit Score','Annual Income','Bankruptcies','Tax Liens']
df = drop_nulls(df, columns)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89586 entries, 0 to 111106
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Loan ID                       89586 non-null  object 
 1   Customer ID                   89586 non-null  object 
 2   Loan Status                   89586 non-null  object 
 3   Current Loan Amount           89586 non-null  int64  
 4   Term                          89586 non-null  object 
 5   Credit Score                  89586 non-null  float64
 6   Years in current job          85799 non-null  object 
 7   Home Ownership                89586 non-null  object 
 8   Annual Income                 89586 non-null  float64
 9   Purpose                       89586 non-null  object 
 10  Monthly Debt                  89586 non-null  float64
 11  Years of Credit History       89586 non-null  float64
 12  Months since last delinquent  42202 non-null  float64
 13  Numbe

### Step 1: Data Cleaning - 4. Drop bad values

In [165]:
def drop_bad_values(data, column_name, bad_value):
  data = data[data[column_name] != bad_value].copy()
  return data

In [166]:
df = drop_bad_values(df, 'Current Loan Amount', 99999999)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76879 entries, 0 to 111106
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Loan ID                       76879 non-null  object 
 1   Customer ID                   76879 non-null  object 
 2   Loan Status                   76879 non-null  object 
 3   Current Loan Amount           76879 non-null  int64  
 4   Term                          76879 non-null  object 
 5   Credit Score                  76879 non-null  float64
 6   Years in current job          73567 non-null  object 
 7   Home Ownership                76879 non-null  object 
 8   Annual Income                 76879 non-null  float64
 9   Purpose                       76879 non-null  object 
 10  Monthly Debt                  76879 non-null  float64
 11  Years of Credit History       76879 non-null  float64
 12  Months since last delinquent  36552 non-null  float64
 13  Numbe

### Step 1: Data Cleaning - 5. Remove Duplicates

In [167]:
def drop_duplicates(data):
  data = data.copy()
  return data.drop_duplicates()

In [168]:
df = drop_duplicates(df);
df.duplicated().sum()

np.int64(0)

In [169]:
df['Loan ID'].duplicated().sum()

np.int64(0)

### Step 1: Data Cleaning - 6. Verify No Duplicates and Set Index to Loan ID

In [170]:
df.set_index('Loan ID', inplace=True)

In [171]:
df.head()

,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
Loan ID,,,,,,,,,,,,,,,,,,
6cf51492-02a2-423e-b93d-676f05b9ad53,7c202b37-2add-44e8-9aea-d5b119aea935,Charged Off,12232,Short Term,7280.0,< 1 year,Rent,46643.0,Debt Consolidation,777.39,18.0,10.0,12,0,6762,7946.0,0.0,0.0
552e7ade-4292-4354-9ff9-c48031697d72,e7217b0a-07ac-47dd-b379-577b5a35b7c6,Charged Off,25014,Long Term,7330.0,10+ years,Home Mortgage,81099.0,Debt Consolidation,892.09,26.7,NaN,14,0,35706,77961.0,0.0,0.0
9b5e32b3-8d76-4801-afc8-d729d5a2e6b9,0a62fc41-16c8-40b5-92ff-9e4b763ce714,Charged Off,16117,Short Term,7240.0,9 years,Home Mortgage,60438.0,Home Improvements,1244.02,16.7,32.0,11,1,11275,14815.0,1.0,0.0
5419b7c7-ac11-4be2-a8a7-b131fb6d6dbe,30f36c59-5182-4482-8bbb-5b736849ae43,Charged Off,11716,Short Term,7400.0,3 years,Rent,34171.0,Debt Consolidation,990.94,10.0,NaN,21,0,7009,43533.0,0.0,0.0
1450910f-9495-4fc9-afaf-9bdf4b9821df,70c26012-bba5-42c0-8dcb-75295ada31bb,Charged Off,9789,Long Term,6860.0,10+ years,Home Mortgage,47003.0,Home Improvements,503.71,16.7,25.0,13,1,16913,19553.0,1.0,0.0


In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64371 entries, 6cf51492-02a2-423e-b93d-676f05b9ad53 to f83ae0e1-bd30-46c7-9f72-f4f9fe2f36ca
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Customer ID                   64371 non-null  object 
 1   Loan Status                   64371 non-null  object 
 2   Current Loan Amount           64371 non-null  int64  
 3   Term                          64371 non-null  object 
 4   Credit Score                  64371 non-null  float64
 5   Years in current job          61577 non-null  object 
 6   Home Ownership                64371 non-null  object 
 7   Annual Income                 64371 non-null  float64
 8   Purpose                       64371 non-null  object 
 9   Monthly Debt                  64371 non-null  float64
 10  Years of Credit History       64371 non-null  float64
 11  Months since last delinquent  29944 non-null  float64
 12 

# --- Step 2: Split train/test/val ---

In [173]:
X = df.drop(columns='Loan Status')
y = df['Loan Status'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [174]:
print(f'Train shape: {X_train.shape}')
print(f'Test shape: {X_test.shape}')
print(f'Test shape: {X_val.shape}')

Train shape: (51496, 17)
Test shape: (6438, 17)
Test shape: (6437, 17)


# --- Step 3: Function Transformers ---

- Credit Score - For all values above 800, remove last 0. 
- Months since last delinquent - change to binary, null as "0", non-null as "1"

### Step 3: Function Transformers - 1. Credit Score
- For all values above 800, remove last 0

In [175]:
df[df['Credit Score'] > 800]['Credit Score'].count()

np.int64(5034)

In [176]:
def adjust_credit(data):
  data = data.copy()
  data.loc[data['Credit Score'] > 800, 'Credit Score'] /= 10
  return data

credit_transformer = FunctionTransformer(adjust_credit)

### Step 3: Function Transformers - 2. Months since last delinquent
- Change to binary, null as "0", non-null as "1"
- Rename column to "Ever Delinquent"

In [177]:
df['Months since last delinquent'].isna().sum()

np.int64(34427)

In [178]:
def delinquent_binary(data):
  data = data.copy()
  data['Months since last delinquent'] = data['Months since last delinquent'].notna().astype(int)
  data.rename(columns={'Months since last delinquent': 'Ever Delinquent'}, inplace=True)
  return data

In [179]:
df = delinquent_binary(df)
df.head(5)

,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Purpose,Monthly Debt,Years of Credit History,Ever Delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
Loan ID,,,,,,,,,,,,,,,,,,
6cf51492-02a2-423e-b93d-676f05b9ad53,7c202b37-2add-44e8-9aea-d5b119aea935,Charged Off,12232,Short Term,7280.0,< 1 year,Rent,46643.0,Debt Consolidation,777.39,18.0,1,12,0,6762,7946.0,0.0,0.0
552e7ade-4292-4354-9ff9-c48031697d72,e7217b0a-07ac-47dd-b379-577b5a35b7c6,Charged Off,25014,Long Term,7330.0,10+ years,Home Mortgage,81099.0,Debt Consolidation,892.09,26.7,0,14,0,35706,77961.0,0.0,0.0
9b5e32b3-8d76-4801-afc8-d729d5a2e6b9,0a62fc41-16c8-40b5-92ff-9e4b763ce714,Charged Off,16117,Short Term,7240.0,9 years,Home Mortgage,60438.0,Home Improvements,1244.02,16.7,1,11,1,11275,14815.0,1.0,0.0
5419b7c7-ac11-4be2-a8a7-b131fb6d6dbe,30f36c59-5182-4482-8bbb-5b736849ae43,Charged Off,11716,Short Term,7400.0,3 years,Rent,34171.0,Debt Consolidation,990.94,10.0,0,21,0,7009,43533.0,0.0,0.0
1450910f-9495-4fc9-afaf-9bdf4b9821df,70c26012-bba5-42c0-8dcb-75295ada31bb,Charged Off,9789,Long Term,6860.0,10+ years,Home Mortgage,47003.0,Home Improvements,503.71,16.7,1,13,1,16913,19553.0,1.0,0.0


In [180]:
delinquent_transformer = FunctionTransformer(delinquent_binary)

# --- Step 4: Imputers ---
1. Years in current job - impute null as constant value "missing data"
2. Maximum Open Credit - impute with median

### Step 4: Imputers - 1. Years in current job 
- impute null as constant value "missing data"

In [181]:
ConstantImputer = SimpleImputer(strategy='constant', fill_value=0)

### Step 4: Imputers - 2. Maximum Open Credit
- Impute with median

In [182]:
MedianImputer = SimpleImputer(strategy='median')

# --- Step 5: Feature Engineering ---
1. Debt to Income Ratio (Monthly Debt / Annual Income)
2. Credit Usage Ratio (Current Credit Balance / Maximum Open Credit)
3. Have Bankruptcies (Binary Yes/No)(Bankruptcies = 0 vs all others)
4. Have Tax Liens (Binary Yes/No)(Tax Liens = 0 vs all others)
5. Have Credit Problems (Binary Yes/No)(Number of Credit Problems = 0 vs all others)

### 1. Debt to Income Ratio (Monthly Debt / Annual Income)

In [183]:
def add_debt_to_income(data):
  data = data.copy()
  data['Debt to Income Ratio'] = data['Monthly Debt'] / data['Annual Income']
  return data

In [184]:
debt_income_transformer = FunctionTransformer(add_debt_to_income)

### 2. Credit Usage Ratio (Current Credit Balance / Maximum Open Credit)

In [185]:
def add_credit_usage(data):
  data = data.copy()
  data['Credit Usage Ratio'] = data['Current Credit Balance'] / data['Maximum Open Credit'].replace(0, 1e-6)
  return data  

In [186]:
credit_usage_transformer = FunctionTransformer(add_credit_usage)

### 3. Have Bankruptcies (Binary Yes/No)(Bankruptcies = 0 vs all others)

In [187]:
def bankruptices_binary(data):
  data = data.copy()
  data['Bankruptcies'] = data['Bankruptcies'].ne(0).astype(int)
  return data

In [188]:
bankruptices_transformer = FunctionTransformer(bankruptices_binary)

### 4. Have Tax Liens (Binary Yes/No)(Tax Liens = 0 vs all others)

In [189]:
def tax_liens_binary(data):
  data = data.copy()
  data['Tax Liens'] = data['Tax Liens'].ne(0).astype(int)
  return data

In [190]:
tax_liens_transformer = FunctionTransformer(tax_liens_binary)

### 5. Have Credit Problems (Binary Yes/No)(Tax Liens = 0 vs all others)

In [191]:
def credit_problems_binary(data):
  data = data.copy()
  data['Number of Credit Problems'] = data['Number of Credit Problems'].ne(0).astype(int)
  return data

In [192]:
credit_problems_transformer = FunctionTransformer(credit_problems_binary)

# --- 5. Encoding & Scaling: ---
Categorical, nominal - OHE
  1. Loan Status
  2. Term
  3. Home Ownership
  4. Purpose

Categorical, ordinal - Ordinal Encoder
  1. Years in current job

Numercial - StandardScaler:
  1. Current Loan Amount
  2. Credit Score
  3. Years of Credit History

Numerical - RobustScaler:
  1. Annual Income
  2. Monthly Debt
  3. Number of Open Accounts
  4. Current Credit Balance
  5. Maximum Open Credit

Feature is ready, no encoding or scaling:
  1. Ever Delinquent
  2. Bankruptcies
  3. Tax Liens
  4. Number of Credit Problems
  5. Debt to Income Ratio
  6. Credit Usage Ratio

In [193]:
categorical_nominal = ['Loan Status', 'Term', 'Home Ownership', 'Purpose']
categorical_ordinal = ['Years in current job']
numeric_standard = ['Current Loan Amount', 'Credit Score', 'Years of Credit History']
numeric_robust = ['Annual Income', 'Monthly Debt', 'Number of Open Accounts', 'Current Credit Balance', 'Maximum Open Credit']
ready_features = ['Ever Delinquent', 'Bankruptcies', 'Tax Liens', 'Number of Credit Problems', 'Debt to Income Ratio', 'Credit Usage Ratio']


preprocessor = ColumnTransformer(transformers=[
  ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_nominal),
  ('ordinal', ce.OrdinalEncoder(mapping = [
    {'col': 'Years in current job', 'mapping': {
      np.nan: 0, '< 1 year': 1, '1 year': 2, '2 years': 3, '3 years': 4, '4 years': 5, 
      '5 years': 6, '6 years': 7, '7 years': 8, '8 years': 9, '9 years': 10, '10+ years': 11
    }}
  ]), categorical_ordinal),
  ('standardscaler', StandardScaler(), numeric_standard),
  ('robustscaler', RobustScaler(), numeric_robust),
  ('passthrough', 'passthrough', ready_features)
], remainder='drop')



In [ ]:
pipeline = Pipeline([
  ('')
])

In [194]:
preprocessor.fit(X_train)
X_train_proc = preprocessor.transform(X_train)
X_test_proc = preprocessor.transform(X_test)

ValueError: A given column is not a column of the dataframe

In [195]:
df.columns

Index(['Customer ID', 'Loan Status', 'Current Loan Amount', 'Term',
       'Credit Score', 'Years in current job', 'Home Ownership',
       'Annual Income', 'Purpose', 'Monthly Debt', 'Years of Credit History',
       'Ever Delinquent', 'Number of Open Accounts',
       'Number of Credit Problems', 'Current Credit Balance',
       'Maximum Open Credit', 'Bankruptcies', 'Tax Liens'],
      dtype='object')